In [ ]:
# %load_ext autoreload
# %autoreload 2

In [2]:
from acemodel import AceModel, Children
from acemodel import default_mtd, corr_rand_mtd
from spew_data.spew import * 

In [3]:
import brfss
from brfss import bfs_data

In [4]:
# scbrfss = bfs_data('./brfss/SCBRFSS.csv')

In [5]:
data_prex = './spew_data/2010_ver1_45079_synth_'
cdl_data = cdn_data(data_prex)
len(cdl_data)

90314

In [6]:
def simulate_stack(mtd, step_num = 1000, display = False):    

    acemodel_list = [
        AceModel(cdl_data[
            (cdl_data[['_RACE_G1', '_INCOMG']] == [r,i]).all(axis = 1)
        ], r, i, step_method = mtd)
        for r in list(brfss.race_list.keys())[1:]
        for i in list(brfss.income_list.keys())[1:]
                    ]
    
    for i in range(step_num):
        [acemodel.step() for acemodel in acemodel_list]

    result = [res for res in [
            acemodel.datacollector.model_vars['Output'][-1]
            for acemodel in acemodel_list]
                if len(res) > 0
            ]
    
    result_df = pd.concat(result, axis = 0, 
                          ignore_index = True)
    
    if not display:
        return result_df
    
    result_bfs = bfs_data(result_df)
    
    for r in list(brfss.race_list.keys())[1:]:
        for i in list(brfss.income_list.keys())[1:]:
            print('RACE: ' + str(brfss.race_list[r]) + ' INCOME: ' + str(brfss.income_list[i]))
            display(result_bfs.get_corr_mat(r,i))
    return result_df

In [7]:
# scbrfss.get_corr_mat(1,1)

In [8]:
corr_model = AceModel(cdl_data[
    (cdl_data[['_RACE_G1', '_INCOMG']] == [1,1]).all(axis = 1)
], 1, 1, step_method = corr_rand_mtd)

In [9]:
for i in range(1000):
    corr_model.step()
    
res = corr_model.datacollector.model_vars['Output'][-1]
bfs_data(res).get_prop_mat(1,1)['pr']

,depress,alcoabuse,drugabuse,prison,patdivorce,phyabuse1,phyabuse2,verbalabuse,sexabuse1,sexabuse2,sexabuse3,foodinsecure
depress,0.214433,0.081787,0.026804,0.017182,0.065979,0.046048,0.033677,0.049485,0.025430,0.007560,0.000687,0.017182
alcoabuse,0.081787,0.331959,0.041237,0.027491,0.083162,0.074914,0.052234,0.081100,0.026804,0.010309,0.002062,0.028179
drugabuse,0.026804,0.041237,0.103093,0.007560,0.028866,0.021306,0.015808,0.026117,0.006873,0.003436,0.000687,0.008935
prison,0.017182,0.027491,0.007560,0.059107,0.021993,0.016495,0.006873,0.012371,0.006873,0.002062,0.000000,0.005498
patdivorce,0.065979,0.083162,0.028866,0.021993,0.257045,0.054296,0.043299,0.064605,0.024742,0.006873,0.000000,0.028179
phyabuse1,0.046048,0.074914,0.021306,0.016495,0.054296,0.202749,0.037113,0.043986,0.023368,0.006186,0.000687,0.015808
phyabuse2,0.033677,0.052234,0.015808,0.006873,0.043299,0.037113,0.137457,0.032302,0.015120,0.005498,0.001375,0.015120
verbalabuse,0.049485,0.081100,0.026117,0.012371,0.064605,0.043986,0.032302,0.216495,0.026804,0.009622,0.002062,0.019931
sexabuse1,0.025430,0.026804,0.006873,0.006873,0.024742,0.023368,0.015120,0.026804,0.082474,0.006186,0.001375,0.010309
sexabuse2,0.007560,0.010309,0.003436,0.002062,0.006873,0.006186,0.005498,0.009622,0.006186,0.019931,0.001375,0.001375


In [10]:
bfs_data(res).get_corr_mat(1,1)

,depress,alcoabuse,drugabuse,prison,patdivorce,phyabuse1,phyabuse2,verbalabuse,sexabuse1,sexabuse2,sexabuse3,foodinsecure
depress,1.000000,0.054864,0.037640,0.046573,0.060552,0.015587,0.029731,0.018108,0.068593,0.057288,0.004550,0.026629
alcoabuse,0.054864,1.000000,0.048986,0.070871,-0.010529,0.040193,0.040725,0.047602,-0.004430,0.056109,0.046609,0.049310
drugabuse,0.037640,0.048986,1.000000,0.020453,0.017809,0.003304,0.015632,0.030325,-0.019482,0.032510,0.025366,0.026124
prison,0.046573,0.070871,0.020453,1.000000,0.065985,0.047579,-0.015416,-0.004377,0.030801,0.026814,-0.013160,0.025670
patdivorce,0.060552,-0.010529,0.017809,0.065985,1.000000,0.012407,0.052942,0.049760,0.029470,0.028646,-0.030883,0.099205
phyabuse1,0.015587,0.040193,0.003304,0.047579,0.012407,1.000000,0.066775,0.000556,0.060093,0.038164,0.006171,0.021352
phyabuse2,0.029731,0.040725,0.015632,-0.015416,0.052942,0.066775,1.000000,0.017937,0.039945,0.057321,0.055282,0.067925
verbalabuse,0.018108,0.047602,0.030325,-0.004377,0.049760,0.000556,0.017937,1.000000,0.078987,0.092195,0.068013,0.051824
sexabuse1,0.068593,-0.004430,-0.019482,0.030801,0.029470,0.060093,0.039945,0.078987,1.000000,0.118130,0.079691,0.068957
sexabuse2,0.057288,0.056109,0.032510,0.026814,0.028646,0.038164,0.057321,0.092195,0.118130,1.000000,0.180345,0.000917


In [ ]:
default_result = simulate_stack(mtd = default_mtd, display = False)

In [ ]:
corr_result = simulate_stack(mtd = corr_rand_mtd, display = False)